# Data Quality and Data Wrangling 
## Course Code: DLBDSDQDW01

## Task 2: Scrape the web

This notebook describes the implementation of Task 2 of the Data Quality and Data Wrangling course (DLBDSDQDW01), it contains the code use for experimentation and the creation of the visualization according to the requirements in the task description.

### Data sources
The data was collected from the following sources:

1. [OpenWeather](https://openweathermap.org/api/one-call-3#concept): for weather data such as temperature, humidity, pressure, etc.
2. [AlphaVantage](https://www.alphavantage.co/documentation/): for Stock market data
2. [USGS Earthquake](https://earthquake.usgs.gov/fdsnws/event/1/): earthquake data for the world

In [ ]:
# used packages
import requests
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
import re
import time
import folium
import os
import json
from dotenv import load_dotenv
from pathlib import Path
from datetime import date, datetime, timezone, timedelta
from zoneinfo import ZoneInfo
from shapely.geometry import Point

In [ ]:
# Load the API keys
load_dotenv(Path().cwd().parent.joinpath(r"config/.env"))

# Define location for time series

In [ ]:
cities = ["Berlin" , "Munich", "Hamburg", "Baden_Baden", "Paris", "Strasbourg", "Madrid", "Malaga", "Barcelona", "Lisbon", "Mexico_city", "Monterrey", "Tokio", "Osaka", "Kyoto"]

# Weather data

Since the API provides data only for one timestamp and not the summary for the whole day, the request will be run every 2 hours, to get 12 readings per day.

In [ ]:
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")

In [ ]:
def fetch(url, headers:dict=None, params:dict=None):
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()

In [ ]:
# geocoding endpoint - to get coordinates of the cities
url_geocoding = "http://api.openweathermap.org/geo/1.0/direct?"
params_geocoding = {"q":None, "limit":1, "appid" : OPENWEATHER_API_KEY}

In [ ]:
responses_cities = []
for city in cities:
    params_geocoding["q"] = city
    response = fetch(url_geocoding, params=params_geocoding)
    responses_cities.append(response)
    time.sleep(1)

In [ ]:
responses_cities

In [ ]:
city_coordinates = {"city" : [], "country" : [], "lon" : [], "lat" : []}
for response_list in responses_cities: # a response list is the response of the API, a list containing a dictionary
    for response in response_list: # response is the dictionary containing the information of the countries
        city_coordinates["city"].append(response["name"])
        city_coordinates["lon"].append(response["lon"]) # X
        city_coordinates["lat"].append(response["lat"]) # Y
        city_coordinates["country"].append(response["country"])
df_geolocations = pd.DataFrame(city_coordinates)
#df_geolocations["geometry"] = df_geolocations.apply(lambda row: Point(row["lon"], row["lat"]), axis=1)
#df_geolocations = gpd.GeoDataFrame(df_geolocations, geometry="geometry", crs="EPSG:4326")
df_geolocations.to_csv("geocoding_openweather.csv", index=False)

In [ ]:
df_geolocations

In [ ]:
df_geolocations = df_geolocations.replace(dict(zip(df_geolocations.city.tolist(), cities)))
df_geolocations

In [ ]:
df_geolocations.explore()

In [ ]:
# weather endpoint - to get weather of the locations
url_weather = "https://pro.openweathermap.org/data/2.5/weather"

In [ ]:
# Time zone for Berlin
berlin = ZoneInfo("Europe/Berlin")

In [ ]:
today = datetime.now(berlin).replace(microsecond=0)

responses_weather = []

for nrow, record in df_geolocations.iterrows():
    lon = record["lon"]
    lat = record["lat"]
    params = {"units":"metric",
              "lon" : lon,
              "lat" : lat,
              "date" : today.isoformat(),
              "appid" : OPENWEATHER_API_KEY}
    response = requests.get(url_weather, params)
    response.raise_for_status()
    responses_weather.append(response)
    time.sleep(1)

In [ ]:
# How does the response look like?
responses_weather[0].json()

In [ ]:
weather = {"name" : [], # city name - it might not match no automatic geocoding by the API
           "temperature" : [], # Temperature
           "temperature_max" : [], # Max temp at the moment
           "temperature_min" : [], # Min temp at the moment
           "feels_like" : [], # Human perception of the weather
           "humidity":[], #
           "wind_speed":[], # in m/
           "wind_direction" : [],
           "description":[],
           "timestamp":[]}

for response in responses_weather:
    weather_data = response.json()
    weather["name"].append(weather_data["name"])
    weather["temperature"].append(weather_data["main"]["temp"])
    weather["temperature_max"].append(weather_data["main"]["temp_max"])
    weather["temperature_min"].append(weather_data["main"]["temp_min"])
    weather["feels_like"].append(weather_data["main"]["feels_like"])
    weather["humidity"].append(weather_data["main"]["humidity"])
    weather["wind_speed"].append(weather_data["wind"]["speed"])
    weather["wind_direction"].append(weather_data["wind"]["deg"])
    weather["description"].append(weather_data["weather"][0]["description"])
    weather["timestamp"].append(pd.to_datetime(weather_data["dt"], unit="s"))
    
df_weather = pd.DataFrame(weather)
df_weather

In [ ]:
# commbine into one dataframe for the weather data
df_weather = pd.concat([df_geolocations, df_weather], axis=1)
df_weather = df_weather.drop("name", axis=1)
df_weather

In [ ]:
df_weather.columns

In [ ]:
df_weather["split_on"] = df_weather["city"]

In [ ]:
df_weather.info()

In [ ]:
# if weather data exists load it, if not
if Path("weather_data.csv").exists():
    print("loading latest data")
    history_df = pd.read_csv("weather_data.csv")
    # this dataframe is the final weather data. Store in staging area to combine later with further data
    df_weather = pd.concat([history_df, df_weather], axis=0).sort_values(by=["city", "timestamp"], ascending=False)
    df_weather.to_csv("weather_data.csv", index=False)
else:
    print("weather_data.csv does not exist. Latest data will be stored")
    df_weather.to_csv("weather_data.csv", index=False)

# Stock market data

In [ ]:
ALPHAVANTAGE_API_KEY = os.getenv("ALPHAVANTAGE_API_KEY")

In [ ]:
#tickers = ("NVDA","AAPL")#,"META","RHM.DE","SPY","URTH","ACWI")
tickers = ("META",)

In [ ]:
tickers

In [ ]:
url_stocks = "https://www.alphavantage.co/query"

In [ ]:
# to build the time series, two cases are identified.
# 1) The first time the VantageClient collects data from the API and 2) with data already exists for the specific symbols

# Does the dataStore already exit?
stocks = []
if Path("/Users/jorgetellez/Documents/06_Projects/IU_Data_Wrangling/data/processed/datastore.h5").exists():
    store = pd.HDFStore("/Users/jorgetellez/Documents/06_Projects/IU_Data_Wrangling/data/processed/datastore.h5", "r")
    print("DataStore content", store.keys())
    keys = [key.split(r"/")[-1] for key in store.keys() if key.split(r"/")[1] == "stocks" and key.split(r"/")[2] == "data"]
    store.close()
    print(keys)
    for symbol in tickers:
        params_stocks = {"function" : "TIME_SERIES_DAILY", # this endpoint provides a daily time series of the equity specified
                     "symbol" : symbol, # the equity -> replace this by all the companies that should be followed
                     "outputsize" : "compact",
                     "dataype" : "json",
                     "apikey" : ALPHAVANTAGE_API_KEY}
        response = requests.get(url_stocks, params=params_stocks)
        response.raise_for_status()
        stock = response.json()["Time Series (Daily)"]
        stocks.append({symbol:stock})

In [ ]:
len(stocks)

In [ ]:
with open(r"/Users/jorgetellez/Documents/06_Projects/IU_Data_Wrangling/data/raw/stocks/20250806_174026_stocks.json") as s:
    stocks = json.load(s)
    print(stocks)
    print(type(stocks))

In [ ]:
stocks[0].keys()

In [ ]:
dfs = []
for stock in stocks:
    for symbol, data in stock.items():
        df = pd.DataFrame(data).T
        df = df.set_index(pd.to_datetime(df.index))
        df = df.apply(pd.to_numeric, errors="coerce")
        df = df.rename(columns={"1. open" :"open", "2. high":"high", "3. low":"low", "4. close":"close", "5. volume": "volume"})
        df["symbol"] = symbol
        df["split_on"] = df["symbol"]
        if symbol in keys:
            df = pd.DataFrame(df.iloc[0, :]).T
            dfs.append(df)
            print(symbol, "exists")
        else:
            dfs.append(df)
df_stocks = pd.concat(dfs, axis=0)
#df_stocks.loc[:,["open", "high", "low", "close", "volume"]] = df_stocks.loc[:,["open", "high", "low", "close", "volume"]].apply(pd.to_numeric, errors="coerce")
df_stocks[["open", "high", "low", "close", "volume"]] = df_stocks[["open", "high", "low", "close", "volume"]].astype("float64")
df_stocks

In [ ]:
df_stocks.dtypes

In [ ]:
fig, ax = plt.subplots(figsize=(24,6), nrows=1, ncols=len(tickers)+1, sharex=True)
for idx, ticker in enumerate(tickers):
    df = df_stocks[df_stocks["symbol"] == ticker]
    ax[idx].set_title(ticker)
    ax[idx].plot(df["open"], marker='.', label="Open")
    ax[idx].plot(df["close"], marker='.', label="Close")
    #ax[idx].set_ylim([0,250])
    ax[idx].grid(True, axis="y", linestyle=":", linewidth=0.5)
    ax[idx].spines[["top", "right", "bottom", "left"]].set_visible(False)
    ax[idx].legend(loc="upper right")

# Earthquake data

Parameters according the API documentation:

In [ ]:
# URL
url = r"https://earthquake.usgs.gov/fdsnws/event/1/"

In [ ]:
# According to the API documentation all times use UTC
# Time in Germany should be specified when making a request
berlin = ZoneInfo("Europe/Berlin")
tokyo = ZoneInfo("Asia/Tokyo")
now = datetime.now(berlin).replace(microsecond=0)
yesterday = now - timedelta(days=5)

In [ ]:
# Query parameters
params = {"method" : "query", # submit a data request
          "format" : "geojson", # reponse format
          #"minlatitude" : 24.0, # Get earthquakes in Japan
          #"maxlatitude" : 46.0,
          #"minlongitude" : 122.0,
          #"maxlongitude" : 146,
          "limit" : 100, # Limit results to this value
          "starttime": yesterday.isoformat(), # the API expect ISO time format, here it is set
          "endtime" : now.isoformat(),
          "orderby" : "time"} # sort the results from most recent to oldest

In [ ]:
# get the data from the API
response = requests.get(url, params=params)
earthquakes = response.json()

In [ ]:
# Use the same names of the reponse in the dict for easier iteration
records = {"time":[], # time when the event ocurred - in milliseconds since the epoch
           "mag":[], # magnitude of the event - combine with magType for interpretation
           "magType":[], # magnitude types are described in the API documentation - must be mapped to a name easier to understand
           "alert":[],
           "tsunami":[],
           "place":[],
           "coordinates":[]}

In [ ]:
for earthquake in earthquakes["features"]:
    for feature in records:
        if feature in earthquake["properties"]:
            records[feature].append(earthquake["properties"][feature])
        else:
            # geometry is a key in the response
            records[feature].append(earthquake["geometry"][feature])

In [ ]:
mag_type_description = {
    "Mw": "Moment Magnitude",
    "Ms": "Surface Wave Magnitude",
    "mb": "Body Wave Magnitude",
    "ml": "Local (Richter) Magnitude",
    "mb_lg": "Lg-Wave Magnitude",
    "md": "Duration Magnitude",
    "MH": "Hand-calculated Magnitude",
    "MI": "Intensity-derived Magnitude",
    "Me": "Energy Magnitude",
    "Mg": "Surface Wave from Ground Displacement",
    "MWb": "Moment Magnitude from Body Waves",
    "Mwr": "Regional Moment Magnitude",
    "MwC": "Centroid Moment Magnitude",
    "MwB": "Body-wave Derived Moment Magnitude",
    "mww": "Moment Magnitude from W-phase"
}


In [ ]:
#TODO: set alert to black in case no alert
df = pd.DataFrame(records)
#df["geometry"] = df.coordinates.apply(lambda coord: Point(coord[:2]))
df["lon"] = df.coordinates.apply(lambda coord: coord[0])
df["lat"] = df.coordinates.apply(lambda coord: coord[1])
df["depth"] = df.coordinates.apply(lambda coord: coord[2])
df = df.rename(columns={"time":"timestamp", "mag":"magnitude", "magType":"scale"})
#df["magType"] = df["magType"].str.lower()
df = df.drop("coordinates", axis=1)

df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
df["split_on"] = df["timestamp"].dt.strftime("date_%Y_%m_%d")
df

In [ ]:
geodf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")
geodf

In [ ]:
geodf.explore()

# Data Storage

In [ ]:
def store(client:str, df:pd.DataFrame, split_on:str):
    # Make sure there is a column on which it can be grouped on
    if split_on not in df.columns:
        raise ValueError(f"{split_on} not found in Dataframe columns: {df.columns}")
    
    # to avoid min_size problems when storing string columns
    for col in df.select_dtypes(include=["object", "string"]).columns:
        entry_len = df[col].astype(str).map(len).max()  
        min_itemsize = max(entry_len, 15) # if the column is empty, use this as the default length

    with pd.HDFStore("/Users/jorgetellez/Documents/06_Projects/IU_Data_Wrangling/data/processed/datastore.h5", "a") as datastore:
        for group, data in df.groupby(split_on):
            datastore.append(
                f"/{client}/data/{group}",
                data,
                format="table",
                data_columns=True,
                min_itemsize=min_itemsize)

In [ ]:
store("weather", df_weather, "split_on")

In [ ]:
store("earthquake", df, "split_on")

In [ ]:
store("stocks", df_stocks, "split_on")

In [ ]:
with pd.HDFStore("/Users/jorgetellez/Documents/06_Projects/IU_Data_Wrangling/data/processed/datastore.h5", "r") as ds:
    for key in ds.keys():
        display(key)
        display(ds[key])
        display()